Importing Libraries

In [ ]:
!pip install scikit-plot

In [ ]:
!pip -q install simpletransformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 250.7/250.7 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 11.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 5.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.7/101.7 kB 13.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 19.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 22.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 25.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.6/190.6 kB 18.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 254.1/254.1 kB 23.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 15.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 18.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 28.2 

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
from simpletransformers.classification import ClassificationModel, ClassificationArgs
import sklearn
from sklearn.metrics import classification_report
from sklearn.metrics import f1_score, accuracy_score
import scikitplot as skplt
import matplotlib.pyplot as plt

In [ ]:
#df = pd.read_csv("Kaggle_Mal_train.csv",sep='\t')
#df_eval = pd.read_csv("Kaggle_Mal_val.csv",sep='\t')
#df_test = pd.read_csv("Kaggle_Mal_test.csv",sep='\t')

In [ ]:
df = pd.read_csv("New_Maltrain_data.csv",sep='\t')
df_eval = pd.read_csv("New_Malval_data.csv",sep='\t')
df_test = pd.read_csv("New_Maltest_data.csv",sep='\t')

In [ ]:
df_eval.rename(columns={0:'text',1:'category'},inplace=True)
df_eval = df_eval[['text','category']]
df.rename(columns={0:'text',1:'category'},inplace=True)
df = df[['text','category']]
df_test.rename(columns={0:'text',1:'category'},inplace=True)
df_test = df_test[['text','category']]

In [ ]:
num_labels = len(df['category'].unique())
keys = list(df['category'].unique())
values = list(range(0, num_labels))
label_dict = dict(zip(keys,values))
df['category'] = df['category'].apply(lambda x:label_dict[x])
df_eval['category'] = df_eval['category'].apply(lambda x:label_dict[x])
df_test['category'] = df_test['category'].apply(lambda x:label_dict[x])
num_labels

11

In [ ]:
df

,text,category
0,"ഉറപ്പ്, ചീര, ചോക്ലേറ്റ് കേക്ക് എന്നിവ ഉൾപ്പെട...",0
1,ജീവിതത്തിലെ വഴിത്തിരിവുകളിലൂടെ നമ്മെ നയിക്കുന്...,1
2,നിങ്ങളുടെ നർമ്മബോധത്തെ അഭിനന്ദിക്കുക; ഇത് അദൃശ...,2
3,തട്ടിൽ പഴയ നെഞ്ച് തുറന്നപ്പോൾ വിന്റേജ് വസ്ത്രങ...,3
4,നമ്മുടെ ദൃഢനിശ്ചയത്തിനും സ്ഥിരോത്സാഹത്തിനും ഊർ...,4
...,...,...
14919,ശരിയായ ആശയവിനിമയം കൂടാെത നിങ്ങളുെട വർക്ക്സോഫ്...,5
14920,അവൻ എന്റെ കൂടെ ഉള്ളത് വരെ ഞാൻ സന്തോഷവതി ആണ്,7
14921,അപ്രതീക്ഷിതമായ ഫയർ അലാറം ആളുകൾ പുറത്തുകടക്കാൻ ...,6
14922,ആർക്കാണ് ഇത് മോശമായതെന്ന് എനിക്കറിയില്ല: കുറ്റ...,0


Balancing the imbalanced dataset

In [ ]:
oversampling_count = 0

In [ ]:
from collections import defaultdict

def oversample(df):
    classes = df['category'].value_counts().to_dict()
    max_count = max(classes.values())
    classes_dict = defaultdict(list)

    # Group data by category
    for index, row in df.iterrows():
        classes_dict[row['category']].append(row['text'])

    oversampled = []

    # Perform oversampling for each class
    for category, texts in classes_dict.items():
        if len(texts) < max_count:
            # Oversample if the class has fewer samples than the maximum count
            oversampled_texts = df[df['category'] == category].sample(n=max_count - len(texts), replace=True)
            oversampled.append(oversampled_texts)

    # Concatenate oversampled data with original DataFrame
    if oversampled:
        df = pd.concat([df] + oversampled, ignore_index=True)

    return df

In [ ]:
print("Oversampling function was called", oversampling_count, "times.")

Oversampling function was called 0 times.


In [ ]:
df=oversample(df)

In [ ]:
df

,text,category
0,"ഉറപ്പ്, ചീര, ചോക്ലേറ്റ് കേക്ക് എന്നിവ ഉൾപ്പെട...",0
1,ജീവിതത്തിലെ വഴിത്തിരിവുകളിലൂടെ നമ്മെ നയിക്കുന്...,1
2,നിങ്ങളുടെ നർമ്മബോധത്തെ അഭിനന്ദിക്കുക; ഇത് അദൃശ...,2
3,തട്ടിൽ പഴയ നെഞ്ച് തുറന്നപ്പോൾ വിന്റേജ് വസ്ത്രങ...,3
4,നമ്മുടെ ദൃഢനിശ്ചയത്തിനും സ്ഥിരോത്സാഹത്തിനും ഊർ...,4
...,...,...
11303,"ദൈവമേ, നമ്മൾ എന്ത് ചെയ്യും? ചെയ്യണോ? വിചിത്രമ",10
11304,"ദൈവമേ, നമ്മൾ എന്ത് ചെയ്യും? ചെയ്യണോ? വിചിത്രമ",10
11305,"ദൈവമേ, നമ്മൾ എന്ത് ചെയ്യും? ചെയ്യണോ? വിചിത്രമ",10
11306,"ദൈവമേ, നമ്മൾ എന്ത് ചെയ്യും? ചെയ്യണോ? വിചിത്രമ",10


In [ ]:
from collections import defaultdict
from sklearn.utils import resample

def over_under_sample(df):
    unq_labels = df['category'].unique()
    data_dict = defaultdict(list)

    # Group texts by category
    for category, text in zip(df['category'], df['text']):
        data_dict[category].append(text)

    # Calculate required length per category
    req_len = len(df) // len(unq_labels)

    new_texts = []
    new_labels = []

    # Iterate through each category in data_dict
    for category, texts in data_dict.items():
        # If category has more samples than required length, truncate
        if len(texts) > req_len:
            texts = texts[:req_len]

        # Oversample to match the length of the category with maximum samples
        if len(texts) < req_len:
            oversampled_texts = resample(texts, replace=True, n_samples=req_len - len(texts))
            texts.extend(oversampled_texts)

        new_texts.extend(texts)
        new_labels.extend([category] * len(texts))

    # Return a DataFrame with oversampled data
    return pd.DataFrame({'text': new_texts, 'category': new_labels})

In [ ]:
df = over_under_sample(df)

In [ ]:
df

,text,category
0,"ഉറപ്പ്, ചീര, ചോക്ലേറ്റ് കേക്ക് എന്നിവ ഉൾപ്പെട...",0
1,തിന്മ ഉത്ഭവിക്കുന്നത് കുറ്റബോധത്തിന്റെ അഭാവത്...,0
2,അറിയാത്ത കുറ്റം സ്വീകരിക്കുന്നതാണ് ഏറ്റവും മോശ...,0
3,കുറ്റബോധം ഒരു നടപടിയും എടുക്കാത്ത ഒരു അലസമായ വ...,0
4,ചിലർ നമ്മുടെ ജീവിതത്തിൽ കിടന്നു വരും വലിയൊരു ന...,0
...,...,...
11303,"ദൈവമേ, നമ്മൾ എന്ത് ചെയ്യും? ചെയ്യണോ? വിചിത്രമ",10
11304,"ദൈവമേ, നമ്മൾ എന്ത് ചെയ്യും? ചെയ്യണോ? വിചിത്രമ",10
11305,"ദൈവമേ, നമ്മൾ എന്ത് ചെയ്യും? ചെയ്യണോ? വിചിത്രമ",10
11306,"ദൈവമേ, നമ്മൾ എന്ത് ചെയ്യും? ചെയ്യണോ? വിചിത്രമ",10


Training the Model

In [ ]:
model_args = ClassificationArgs()

In [ ]:
model_args.overwrite_output_dir=True
model_args.eval_batch_size=8
model_args.train_batch_size=8
model_args.learning_rate=4e-5
model_args.num_train_epochs = 4

In [ ]:
model = ClassificationModel(
    'bert',
    'google/muril-base-cased',
    num_labels=12,
    args=model_args,
    tokenizer_type="bert",
    tokenizer_name='google/muril-base-cased'
)

config.json:   0%|          | 0.00/411 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/953M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at google/muril-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tokenizer_config.json:   0%|          | 0.00/206 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/3.16M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/113 [00:00<?, ?B/s]

In [ ]:
for i in range(0,3):
    !rm -rf /kaggle/working/outputs
    model.train_model(df,eval_data=df_eval,acc=sklearn.metrics.classification_report)
    result, model_outputs, preds_list = model.eval_model(df_test,acc=sklearn.metrics.classification_report)
    for j in result.values():
        print(j)

/usr/local/lib/python3.10/dist-packages/simpletransformers/classification/classification_model.py:611: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  warnings.warn(


  0%|          | 0/11308 [00:00<?, ?it/s]

Epoch:   0%|          | 0/4 [00:00<?, ?it/s]

Running Epoch 0 of 4:   0%|          | 0/1414 [00:00<?, ?it/s]

Running Epoch 1 of 4:   0%|          | 0/1414 [00:00<?, ?it/s]

...

/usr/local/lib/python3.10/dist-packages/simpletransformers/classification/classification_model.py:611: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  warnings.warn(


  0%|          | 0/11308 [00:00<?, ?it/s]

Epoch:   0%|          | 0/4 [00:00<?, ?it/s]

Running Epoch 0 of 4:   0%|          | 0/1414 [00:00<?, ?it/s]

Running Epoch 1 of 4:   0%|          | 0/1414 [00:00<?, ?it/s]

In [ ]:
predictions, raw_outputs = model.predict(df_test['text'].to_list())

In [ ]:
df_final = df_test.copy()
reverse_label_dict = {v:u for u,v in label_dict.items()}
reverse_label_dict
df_final['Predicted_Labels'] = predictions
df_final['Predicted_Labels'] = df_final['Predicted_Labels'].apply(lambda x:reverse_label_dict[x])
df_final['category'] = df_final['category'].apply(lambda x:reverse_label_dict[x])
df_final['pid'] = df_final.index
df_final = df_final[['pid','Predicted_Labels','category']]

In [ ]:
df_final



In [ ]:
score = f1_score(df_final['category'],df_final['Predicted_Labels'],average='macro')
print("The macro average f1 score is:" + str(score))

In [ ]:
accuracy= accuracy_score(df_final['category'],df_final['Predicted_Labels'])
print("The accuracy is:" + str(accuracy))

In [ ]:
import pandas as pd
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score


In [ ]:
# Specify the number of folds
n_splits = 5  # You can adjust the number of folds as needed

# Initialize StratifiedKFold
stratified_kfold = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)

In [ ]:
# Initialize a metric (e.g., accuracy) to keep track of performance across folds
overall_accuracy = 0.0


In [ ]:
# Perform cross-validation
for fold, (train_index, val_index) in enumerate(stratified_kfold.split(df_eval['text'], df_eval['category'])):
    print(f"\nFold {fold + 1}:")

# Split the data into training and validation sets
    train_data, val_data = df_eval.iloc[train_index], df_eval.iloc[val_index]

    # Train the model
   # model.train_model(train_data)

    # Evaluate on the validation set
    predictions, raw_outputs  = model.predict(val_data['text'].to_list())
    accuracy = accuracy_score(val_data['category'], predictions)

    print(f"Accuracy: {accuracy:.4f}")

    # Aggregate performance metric
    overall_accuracy += accuracy


In [ ]:
# Calculate average accuracy
accuracy_scores = [0.8342,  0.8472, 0.8445, 0.8606, 0.8552]
#average_accuracy = sum(accuracy_scores) / len(accuracy_scores)
average_accuracy = sum(accuracy_scores) / n_splits
print(f"Average Accuracy: {average_accuracy}")

In [ ]:
# Assuming 'model' is your trained ClassificationModel
# Replace 'model' with your actual model variable name
model.save_model("/content/drive/My Drive/Emotion_Model_Directory")  # Save model

# Save tokenizer separately
model.tokenizer.save_pretrained("/content/drive/My Drive/Emotion_Model_Directory")  # Save tokenizer

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Assuming df_final contains the DataFrame with 'Predicted_Labels' and 'category' columns

# Compute misclassifications
df_final['Misclassified'] = df_final['Predicted_Labels'] != df_final['category']

# Aggregate misclassification counts
error_counts = df_final.groupby(['category', 'Misclassified']).size().unstack().fillna(0)


In [ ]:
# Plotting graph for error analysis
error_counts.plot(kind='bar', stacked=True, figsize=(5, 6))
plt.xlabel('Categories')
plt.ylabel('Count')
plt.title('Misclassification Count by Category with MuRIL model')
plt.legend(title='classification', labels=['Correct', 'Misclassified'])
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()
